# 03 - Model Deployment

The purpose of this notebook is to execute a CI/CD routine to test and deploy the trained model to AI Platform Prediction, using Cloud Build.
The CI/CD rountine is defined in the [model-deployment.yaml](model-deployment.yaml) file, and consists of the following steps:
1. Load and test the the trained model interface.
2. Create and endpoint in AI Platform Prediction if it doesn't exists.
3. Deploy the model to the endpoint.
4. Test the endpoint.
